In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Libraries

In [ ]:
from datetime import datetime
from datetime import date

# Explore Data

In [ ]:
df = pd.read_csv('/kaggle/input/open-shopee-code-league-logistic/delivery_orders_march.csv')
print("Shape: {}".format(df.shape))
df

In [ ]:
df_mat = pd.read_excel('/kaggle/input/open-shopee-code-league-logistic/SLA_matrix.xlsx')
print("Shape: {}".format(df_mat.shape))
df_mat

# Define Data (and Preprocess)

In [ ]:
sla_attempt_1 = [[3,5,7,7],
                 [5,5,7,7],
                 [7,7,7,7],
                 [7,7,7,7]]

sla_attempt_2 = [[3,3,3,3],
                 [3,3,3,3],
                 [3,3,3,3],
                 [3,3,3,3]]

workdays_list = '1111110'

holidays_list = ['2020-03-08', '2020-03-25', '2020-03-30', '2020-03-31']

city = ['Metro Manila', 'Luzon', 'Visayas', 'Mindanao']

In [ ]:
%%time

def extract_city(s):
    for i in range(len(city)):
        if city[i].lower() in s.lower():
            return i

def address_to_city_code():
    df['buyeraddress'] = [extract_city(x) for x in df['buyeraddress']]
    df['selleraddress'] = [extract_city(x) for x in df['selleraddress']]

address_to_city_code()

In [ ]:
%%time

def extract_time(t):
    if not pd.isnull(t):
        tmp = datetime.fromtimestamp(t)
        tmp = tmp.replace(hour=0, minute=0, second=0)
        return tmp.strftime('%Y-%m-%d')
    else:
        return ''
    
def timestamp_to_date():
    df['pick'] = [extract_time(x) for x in df['pick']]
    df['1st_deliver_attempt'] = [extract_time(x) for x in df['1st_deliver_attempt']]
    df['2nd_deliver_attempt'] = [extract_time(x) for x in df['2nd_deliver_attempt']]

timestamp_to_date()

In [ ]:
df

# Process

In [ ]:
%%time

bdd = np.busdaycalendar(weekmask=workdays_list, holidays=holidays_list)

def check_is_late(inp):
    count_workdays = np.busday_count(inp[2], inp[3], busdaycal=bdd)-1
    tmp = count_workdays >= sla_attempt_1[inp[6]][inp[5]]
    
    if inp[4] == '':
        return int(tmp)
    else:
        count_workdays = np.busday_count(inp[3], inp[4], busdaycal=bdd)-1
        return int(tmp or (count_workdays >= sla_attempt_2[inp[6]][inp[5]]))

df['is_late'] = [check_is_late(x) for x in df.itertuples()]

In [ ]:
df

# Result

In [ ]:
submission = df[['orderid','is_late']].copy()
submission.to_csv('submission.csv', index=False)

In [ ]:
submission